In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import statsmodels.api as sm

plt.rcParams['font.family'] = 'HYGraphic-Medium'

In [25]:
df = {}

for i in range(2014, 2022):
    df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
    df[i].drop(labels=df[i][df[i]['아내지역코드'] == 80].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편지역코드'] == 80].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내직업코드'] == 99].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내결혼코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편결혼코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편교육코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내교육코드'] == 9].index, inplace=True)

mandic = {1:10015,2:5011,3:4833,5:3325.5,6:2582,7:3510,8:3539,9:2494,13:0}
womandic = {1:8024,2:3136,3:3256,5:2119,6:2060,7:2156,8:2795,9:1897,13:0}

for i in range(2014,2022):
    df[i]['남편소득수준'] = df[i]['남편직업코드']
    df[i]['아내소득수준'] = df[i]['아내직업코드']
    
    for j in mandic.keys():
        df[i]['남편소득수준'].replace(j,mandic[j], inplace=True)
        df[i]['아내소득수준'].replace(j,womandic[j], inplace=True)

df[2019] = pd.concat([df[2014], df[2015], df[2016], df[2017], df[2018], df[2019], df[2020], df[2021]])

C:\Users\nexen\AppData\Local\Temp\ipykernel_29528\3270219976.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\nexen\AppData\Local\Temp\ipykernel_29528\3270219976.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\nexen\AppData\Local\Temp\ipykernel_29528\3270219976.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\nexen\AppData\Local\Temp\ipykernel_29528\3270219976.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")


In [26]:
year = 2019
# 소득  (연속)

# 소득 차이 (연속)

# 여성결혼연령 (연속)



# 연상혼
df[year].loc[df[year]['남편연령'] < df[year]['아내연령'], "연상혼여부"] = 1
df[year].loc[df[year]['남편연령'] >= df[year]['아내연령'], "연상혼여부"] = 0

# 아내 대도시
df[year].loc[df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), "아내대도시여부"] = 1
df[year].loc[~df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), "아내대도시여부"] = 0

# 남편 대도시
df[year].loc[df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]), "남편대도시여부"] = 1
df[year].loc[~df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]), "남편대도시여부"] = 0

# 남자 결혼 종류

# 여자 결혼 종류

# 아내 초혼 여부
df[year].loc[df[year]['아내결혼코드'] == 1, "아내초혼여부"] = 1
df[year].loc[df[year]['아내결혼코드'] != 1, "아내초혼여부"]= 0

# 남편 초혼 여부
df[year].loc[df[year]['남편결혼코드'] == 1, "남편초혼여부"] = 1
df[year].loc[df[year]['남편결혼코드'] != 1, "남편초혼여부"]= 0

# 결혼 종류 차이 여부
df[year].loc[df[year]['남편초혼여부'] == df[year]['아내초혼여부'], "결혼종류차이여부"] = 1
df[year].loc[df[year]['남편초혼여부'] != df[year]['아내초혼여부'], "결혼종류차이여부"]= 0

# 강혼 유무
df[year].loc[df[year]['아내교육코드'] > df[year]['남편교육코드'], "강혼여부"] = 1
df[year].loc[df[year]['아내교육코드'] <= df[year]['남편교육코드'], "강혼여부"] = 0

# 소득 차
df[year]['소득차'] = df[year]['아내소득수준'] - df[year]['남편소득수준']

df[year]['아내연령제곱'] = df[year]['아내연령'] ** 2
df[year]['남편연령제곱'] = df[year]['남편연령'] ** 2

# 연상혼 유무
df[year].loc[df[year]['아내연령'] > df[year]['남편연령'], "연상혼여부"] = 1
df[year].loc[df[year]['아내연령'] <= df[year]['남편연령'], "연상혼여부"] = 0

# 아내 무직 여부
df[year].loc[df[year]['아내직업코드'] == 13, "아내무직여부"] = 1
df[year].loc[df[year]['아내직업코드'] != 13, "아내무직여부"] = 0

# 남편 무직 여부
df[year].loc[df[year]['남편직업코드'] == 13, "남편무직여부"] = 1
df[year].loc[df[year]['남편직업코드'] != 13, "남편무직여부"] = 0

df[year].loc[df[year]['남편지역코드'] == df[year]['아내지역코드'], "동일지역여부"] = 1
df[year].loc[df[year]['남편지역코드'] != df[year]['아내지역코드'], "동일지역여부"] = 0

df[year].loc[df[year]['아내교육코드'] == 4, "아내고졸여부"] = 1
df[year].loc[df[year]['아내교육코드'] != 4, "아내고졸여부"] = 0

df[year].loc[df[year]['아내교육코드'] == 5, "아내대졸여부"] = 1
df[year].loc[df[year]['아내교육코드'] != 5, "아내대졸여부"] = 0

df[year].loc[df[year]['아내교육코드'] == 6, "아내대학원졸여부"] = 1
df[year].loc[df[year]['아내교육코드'] != 6, "아내대학원졸여부"] = 0

df[year].loc[df[year]['아내지역코드'].isin([11, 23, 31]), "아내수도권여부"] = 1
df[year].loc[~df[year]['아내지역코드'].isin([11, 23, 31]), "아내수도권여부"] = 0

df[year].loc[df[year]['남편지역코드'].isin([11, 23, 31]), "남편수도권여부"] = 1
df[year].loc[~df[year]['남편지역코드'].isin([11, 23, 31]), "남편수도권여부"] = 0

df[year].loc[df[year]['실제결혼연도'] == 2015, "15년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2015, "15년여부"] = 0
df[year].loc[df[year]['실제결혼연도'] == 2016, "16년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2016, "16년여부"] = 0
df[year].loc[df[year]['실제결혼연도'] == 2017, "17년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2017, "17년여부"] = 0
df[year].loc[df[year]['실제결혼연도'] == 2018, "18년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2018, "18년여부"] = 0
df[year].loc[df[year]['실제결혼연도'] == 2019, "19년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2019, "19년여부"] = 0
df[year].loc[df[year]['실제결혼연도'] == 2020, "20년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2020, "20년여부"] = 0

df[year].loc[df[year]['실제결혼연도'] == 2021, "21년여부"] = 1
df[year].loc[df[year]['실제결혼연도'] != 2021, "21년여부"] = 0

woman_int_condition = (df[year]['남편국적구분코드'] == 1) & df[year]['아내국적구분코드'].isin([2,3])

# df[year].loc[woman_int_condition, "아내국제결혼여부"] = 1
# df[year].loc[~woman_int_condition, "아내국제결혼여부"] = 0


In [27]:
# ct = pd.crosstab(df[year]['아내국제결혼여부'], df[year]['강혼여부'])

# ct

### 출생코호트 -> 5년 단위
### 출생연도 기준 코호트 : 출생연도 계산 [실결혼연도 - 연령] -> 연령 올림후 연도에서 빼기

In [28]:
import math

df[year]['아내연령올림'] = df[year]['아내연령'].apply(np.ceil)

df[year]['아내출생연도'] = df[year]['실제결혼연도'] - df[year]['아내연령올림']


In [29]:
#아내 출생코호트

min = 1970
step = 5
for y in range (1970, 2000, step):
    df[year].loc[df[year]['아내출생연도'].between(y, y + 5), "아내코호트"] = f"{y} to {y + 5}" 

for code in df[year]['아내코호트'].unique():
    df[year].loc[df[year]['아내코호트'] == code, str(code) + " (아내)"] = 1
    df[year].loc[df[year]['아내코호트'] != code, str(code) + " (아내)"] = 0



In [30]:
df[year].drop(axis = 1, labels='nan (아내)', inplace=True)

In [31]:
#남편 출생연도
import math

df[year]['남편연령올림'] = df[year]['남편연령'].apply(np.ceil)

df[year]['남편출생연도'] = df[year]['실제결혼연도'] - df[year]['남편연령올림']

#남편 출생코호트

min = 1970
step = 5

for y in range (1970, 2000, step):
    df[year].loc[df[year]['남편출생연도'].between(y, y + 5), "남편코호트"] = f"{y} to {y + 5}" 

for code in df[year]['남편코호트'].unique():
    df[year].loc[df[year]['남편코호트'] == code, str(code) + " (남편)"] = 1
    df[year].loc[df[year]['남편코호트'] != code, str(code) + " (남편)"] = 0

In [32]:
df[year].drop(axis = 1, labels='nan (남편)', inplace=True)

In [43]:
features = df[year][['동일지역여부','남편무직여부','아내무직여부','연상혼여부','아내연령제곱',
'아내대도시여부','남편대도시여부', '아내고졸여부', '아내대졸여부', '아내대학원졸여부', '아내수도권여부','아내연령',
 '1970 to 1975 (아내)','1975 to 1980 (아내)','1980 to 1985 (아내)', '1985 to 1990 (아내)','1990 to 1995 (아내)','1995 to 2000 (아내)',
'1970 to 1975 (남편)', '1975 to 1980 (남편)','1980 to 1985 (남편)','1985 to 1990 (남편)','1990 to 1995 (남편)','1995 to 2000 (남편)']]
hypogamy = df[year]['강혼여부']
scaler = StandardScaler()
features = scaler.fit_transform(features)
model = sm.Logit(hypogamy,features)
results = model.fit()
print("aic : " + results.aic.__str__())
print('강혼으로 로지스틱 회귀분석')
print(results.summary(xname=['동일지역여부','남편무직여부','아내무직여부','연상혼여부','아내연령제곱',
'아내대도시여부','남편대도시여부', '아내고졸여부', '아내대졸여부', '아내대학원졸여부', '아내수도권여부',  '아내연령',
 '1970 to 1975 (아내)','1975 to 1980 (아내)','1980 to 1985 (아내)', '1985 to 1990 (아내)','1990 to 1995 (아내)','1995 to 2000 (아내)',
'1970 to 1975 (남편)', '1975 to 1980 (남편)','1980 to 1985 (남편)','1985 to 1990 (남편)','1990 to 1995 (남편)','1995 to 2000 (남편)']))

# features = df[year][['동일지역 여부','남편 무직 여부','아내 무직 여부','연상혼 여부','아내연령제곱','소득차','결혼종류 차이 유무','아내 초혼 여부','남편 초혼 여부',
# '아내 대도시 유무','남편 대도시 유무','남편소득수준','아내소득수준', '아내연령', '남편연령', '남편연령제곱', 
# '1970 to 1975 (아내)','1975 to 1980 (아내)','1980 to 1985 (아내)', '1985 to 1990 (아내)','1990 to 1995 (아내)','1995 to 2000 (아내)',
# '1970 to 1975 (남편)', '1975 to 1980 (남편)','1980 to 1985 (남편)','1985 to 1990 (남편)','1990 to 1995 (남편)','1995 to 2000 (남편)']]

Optimization terminated successfully.
         Current function value: 0.632586
         Iterations 7
aic : 2256733.5224455516
강혼으로 로지스틱 회귀분석
                           Logit Regression Results                           
Dep. Variable:                   강혼여부   No. Observations:              1783699
Model:                          Logit   Df Residuals:                  1783675
Method:                           MLE   Df Model:                           23
Date:                Mon, 15 Aug 2022   Pseudo R-squ.:                 -0.3659
Time:                        15:00:50   Log-Likelihood:            -1.1283e+06
converged:                       True   LL-Null:                   -8.2608e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
동일지역여부                0.0305      0.002     18.871    

In [44]:
features = df[year][['동일지역여부','남편무직여부','아내무직여부','연상혼여부','아내연령제곱',
'아내대도시여부','남편대도시여부', '아내고졸여부', '아내대졸여부', '아내대학원졸여부', '아내수도권여부',  '아내연령',
 '15년여부', '16년여부', '17년여부', '18년여부', '19년여부', '20년여부', '21년여부']]
hypogamy = df[year]['강혼여부']
scaler = StandardScaler()
features = scaler.fit_transform(features)
model = sm.Logit(hypogamy,features)
results = model.fit()
print("aic : " + results.aic.__str__())
print('강혼으로 로지스틱 회귀분석')
print(results.summary(xname=['동일지역여부','남편무직여부','아내무직여부','연상혼여부','아내연령제곱',
'아내대도시여부','남편대도시여부', '아내고졸여부', '아내대졸여부', '아내대학원졸여부', '아내수도권여부',  '아내연령',
 '15년여부', '16년여부', '17년여부', '18년여부', '19년여부', '20년여부', '21년여부']))

Optimization terminated successfully.
         Current function value: 0.635342
         Iterations 7
aic : 2266554.4012984005
강혼으로 로지스틱 회귀분석
                           Logit Regression Results                           
Dep. Variable:                   강혼여부   No. Observations:              1783699
Model:                          Logit   Df Residuals:                  1783680
Method:                           MLE   Df Model:                           18
Date:                Mon, 15 Aug 2022   Pseudo R-squ.:                 -0.3718
Time:                        15:02:56   Log-Likelihood:            -1.1333e+06
converged:                       True   LL-Null:                   -8.2608e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
동일지역여부         0.0229      0.002     14.344      0.000       0.020  